# Prevendo o imprevisível: agregação de pesquisas eleitorais em 2022

Se você estiver lendo a versão interativa desta apresentação no binder, pressione `espaço` para mover para o próximo slide.

# Para aqueles executando localmente
Remova todos headers %%capture para obter as imagens dos resultados

In [1]:
import matplotlib.pyplot as plt
from scipy import stats, optimize
import numpy as np
import pandas as pd
import jotaviz
from jotaviz import styles
from datetime import date, datetime, timedelta
np.set_printoptions(precision=2, floatmode="fixed")

In [2]:
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)

## Apresentação

Esta apresentação será apresentada de uma forma que não exigirá conhecimento profundo de nenhum dos tópicos técnicos, quero apenas mostrar a intuição, problemas, agregação e modelos de projeção.

### Introdução
   * Por que as pesquisas variam tanto?
   * Data is now everywhere is well
   * Data informed decisions not so much

### Decisions from summary analysis
   * This is what most businesses do using Excel or Tableau
        *Nothing wrong with this approach. Its quick and reliable and most folks have the skillset
   * While data focused the approach leads itself to human bias
   *  Leaves information on the table
   * Not very reproducible
   * Doesn't scale to many dimensions
   * We can do better

### Loss/Cost/Reward/Utility Functions

    * Taking an idea from Game Theory different versions of the future will have different goodness
    * This lets us determine how much reward we'll get for different actions

### Quantifying Uncertainty

   * Bayesian Methods let us predict the future, not just summarize the past
   * Not just one future though but all futures
   * Particularly well suited to explaining why its predictions are the way they are

### Optimizing over all possible futures

    * Given a distribution of possible futures and a cost function we can optimize to pick an action that will most likely result in the best outcome
    * Open Source code makes it really easy these days!


Technical Summary of talk
I'll be presenting most likely using Jupyter slides. The analysis will cover a little bit of supply chain theory in math form, and then a whole bunch of simulated supply chain theory in using scipy.stats.(random) methods, PyMC3 or Stan for parameter estimation, more scipy for optimization methods, matplotlib for plotting. Of course numpy and pandas will likely be in there somewhere as well.

# Quem sou eu?
* Daniel Marcelino
* Analista de dados no JOTA em Brasília
* Twitter e github: dmarcelinobr
* E-mail: daniel.marcelino@jota.info


<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/template-colunistas-pro-21.png?raw=true" alt="Crédito: Raoni Arruda" style="height: 180px;"/> </right>

# As pesquisas eleitorais...
## Para as consultorias e institutos de opinião as pesquisas funcionam assim:


<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/target-with-dart-flat.jpg?raw=true" alt="" style="height: 300px;"/> </right>

# As pesquisas eleitorais...
## Mas na prática... bem, elas funcionam mais ou menos assim:


<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/archery-target-ring.png?raw=true" alt="" style="height: 400px;"/> </right>

# Teoria vs. prática

#### René Descartes
- Toda explicação científica se deve expressar em termos de quantidades precisas e matematicamente definidas (dúvida cartesiana).

#### Carl Hempel
- As ciências são modelos da realidade, explanatórios e preditivos, em aproximações sucessivas da realidade estudada. 

#### Thomas Kuhn
- A ciência se desenvolve através de paradigmas que se sucedem na explicação dos fatos.

É por isso que a Mecânica Newtoniana é suficiente para o lançamento de satélites e previsão da órbita dos planetas (exceto Mercúrio).

# Teoria vs. prática

* As pesquisas eleitorais refletem uma dada opinião em um certo momento. O Pai-Nosso das consultorias diz que: **"é como uma fotografia!"** 

* Por se tratar de opinião, matéria-prima bastante volátil, o grau de variação entre um momento e outro também está envolto em um quadro móvel de circunstâncias.

* Qualquer informação nova pode ou não alterar esse quadro.


# As pesquisas são incertas por natureza

* Todo modelo científico se **respalda na explicação dos resultados** e na **previsão** do que possa vir a ocorrer. Assim é na física, na química e na biologia, dentre alguns graus possíveis de previsibilidade.

* Na estatística, os modelos são de menor previsibilidade do que nas ciências físicas, devido à **margem de erro** ou de variação aleatória inerente ao método amostral. 

* Lembre-se: as pesquisas baseiam-se na teoria estatística da **representatividade** de uma amostra, dentro de um universo. Então o que ocorre quando podemos ter várias representações dessa realidade?

# As pesquisas são incertas por natureza

* Nas pesquisas eleitorais, os modelos são de **menor previsibilidade ainda**, devido à margem de erro e à mutação contínua das opiniões e do voto. 

* Mas é importante observar outros aspectos, inclusive linguísticos contidos na expressão **margem de erro**. 

* Por exemplo, em português, o termo **erro** é carregado de conotação negativa. Já em inglês, o termo **error**, utilizado na expressão **margin of error**, difere do termo **mistake**, que traz a conotação negativa na língua inglesa. 

* A **margem de erro**, ou **margin of error**, deve ser entendida como **margem de variação técnica** antecipadamente prevista em uma amostra, e não como **erro** ocorrido após uma medição.

# O mundo real é mais complexo
### Parece difícil medir intenção de votos para Lula

<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/voto_espontaneo_lula.png?raw=true" alt="Daniel" style="height: 400px;"/> </right>

# O mundo real é mais complexo
### Parece difícil medir a intenção de votos para Bolsonaro também 

<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/voto_espontaneo_bolsonaro.png?raw=true" alt="Daniel" style="height: 400px;"/> </right>

# A teoria funciona, mas... 

* Eleitores não são como descritos em experimentos de probabilidade nos livros didáticos de estatística, realizados com bolas coloridas que nunca mundam de cor.

* No mundo real eleitoras e eleitores podem ou não mudar de opinião, de acordo com estímulos que recebem. As campanhas estão aí pra isso! 

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/porcentagem.gif?raw=true" alt="mutatis mutandis" style="height: 300px;"/> </center>

# A sulução é modelar os dados de opinião
### Modelos 

* Na minha experiência, modelos bayesianos são particularmente poderosos em caracterizar a incerteza, mesmo em situações com uma pequena quantidade de dados. 

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/auxilio_brasil_model_1T-VT.png?raw=true" alt="mutatis mutandis" style="height: 300px;"/> </center>

# Exemplo de dados 

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/dados_agregadores.png?raw=true" alt="mutatis mutandis" style="height: 500px;"/> </center>

# An example: Making decisions as newspaper vendor

## Anisha is starting her new enterprise

* She buys them wholesale for 5 dollars and sells them for 7 dollars. 
* Newspapers only last one day, they become worthless the next day. 
* Each morning she must make a choice of how many to purchase. 
* If she buys too little she loses an opportunity to sell, if she buys too many she wastes money on unsold papers. 

How many should she purchase in the morning to **maximize her profit**?

# How can Anisha get the best outcome using math?
This problem is a "textbook example" Operations Research problem called the newsvendor problem. Also called Single period model


# Decision 1. Anisha buys 42 newspapers
Let's assume the demand is fixed at 40 newspapers

In [3]:
newspaper_cost = 5
customer_price = 7

anishas_inventory = 42
demand = 40

profit = demand*customer_price - newspaper_cost*anishas_inventory
 
f"Anisha makes ${profit} in profit"

'Anisha makes $70 in profit'

# Decision 2. Anisha buys 38 newspapers
The demand is 40 newspapers but Anisha can only sell as many as she bought

In [4]:
newspaper_cost = 5
customer_price = 7
anishas_inventory = 38

# Anisha can't sell more news papers than she can buy so she sells out
profit = anishas_inventory*customer_price - newspaper_cost*anishas_inventory
f"Anisha makes ${profit} in profit"

'Anisha makes $76 in profit'

# Let's make this a function

In [5]:
def profit_calculator(inventory, demand, cogs=5, price=7):
    """Loss function for unmet demand"""
    profit_per_sale = price-cogs
    if demand < inventory:
        leftover_product = inventory-demand
        
        # I'm unhappy because I lost money on unsold inventory
        utility = demand*profit_per_sale - leftover_product * cogs
    
    if demand >= inventory:        
        # I'm unhappy because I could have sold more newspapers
        utility =  inventory*profit_per_sale 
        
    return utility

In [6]:
f"Decision 1: profit ${profit_calculator(42, 40)}, Decision 2 profit ${profit_calculator(38, 40)}"

'Decision 1: profit $70, Decision 2 profit $76'

In [9]:
%%capture
choices_anisha_could_make = list(range(100))
profit_with_each_choice = [profit_calculator(inv, 40) for inv in choices_anisha_could_make]

fig, ax=plt.subplots(figsize=(10,6))
ax.plot(choices_anisha_could_make, profit_with_each_choice)
ax.axvline(40, linestyle='--', c="gray")
ax.set_xlabel("Number of Newspaper Anisha Bought")
ax.set_title(("Amount of profit Anisha makes in Dollars if demand is fixed at 40 Units \n"
              "Rapid dropoff is because Anisha makes 2 dollars on each paper sold \n"
              "but loses 5 dollars on each unsold")
            )
fig.savefig("img/ProfitCurve.png")

#  Profit curve for all the choices
Let's plot the profit across inventory choices from 0 to 100

In [10]:
choices_anisha_could_make = list(range(100))
profit_with_each_choice = [profit_calculator(inv, 40) for inv in choices_anisha_could_make];

<center><img src="img/ProfitCurve.png" alt="Drawing" style="height: 700px;"/> </center>

# But how sure are we?
We have 5 data points, do we confident observed mean and standard deviation are the *true* mean and standard deviation? 

## With a small amount of data we can't be sure
Yet with the critical fractile we can only choose one mean and one standard deviation to optimize against

Unfortunately with such little data the estimate of the **true mean** and **true standard deviation** can be challenging

# Simulations!

In [17]:
simulations = 100
mean, std = [], []

for _ in range(simulations):
    true_mean,true_std = 40, 20
    values = stats.norm(40, 20).rvs(5)
    mean.append(values.mean())
    std.append(values.std())

<center><img src="img/Uncertainty2.jpg" alt="Drawing"/> </center>

# Methodology 3: Let's use Bayesian Statistics
Bayesian statistics treats the data as fixed, but the parameters like mean and standard deviation as unknown. Let's first estimate our parameters.

# Distributions estimates of the Mean and Standard deviation of True demand distribution
In Bayesian statistics, one of the outputs are posterior distributions of parameters that describe *possible* distributions of newspapers sold each day  

In our case $N(\mu, \sigma)$ , or "Normal centered at *mu* with standard deviation *sigma*"

# Muito obrigado pela presença!

In [18]:
# Set seed back to zero for rest of notebook
ELECTION_DATE = datetime.datetime(year=2022, month=10, day=2)
TODAY = date.today().isoformat()

countdown = ELECTION_DATE - datetime.datetime.now()

print(f"Countdown to Election BR 2022: {countdown}")